# Python to Javascript construct mapping tests

Test that expressions evaluated for widgets in the Python context
do something similar to what they would do in Javascript context

In [ ]:
import jp_proxy_widget

from jp_proxy_widget import notebook_test_helpers

validators = notebook_test_helpers.ValidationSuite()

In [ ]:
class Callback:
    
    def __init__(self):
        self.called = False
        
    def __call__(self, argument=True):
        self.called = argument
    

# Call setTimeout with callback

In [ ]:
import time
t_start = time.time()
t_end = None

def t_callback():
    global t_end
    t_end = time.time()
    t_widget.element.html("Start and end: " + repr((t_start, t_end, t_end-t_start)))
    
def t_validate():
    assert t_end is not None, "no t_end"
    assert (t_end - t_start) > 0.5, repr(("bad timing", t_start, t_end, t_end - t_start))
    print ("setTimeout ok.")
    
t_widget = jp_proxy_widget.JSProxyWidget()
t_widget.debugging_display()

In [ ]:
t_widget.setTimeout(t_callback, 500)  # half a second

validators.add_validation(t_widget, t_validate)

# Get the page title

In [ ]:
pt_widget = jp_proxy_widget.JSProxyWidget()
pt_callback = Callback()
pt_widget.save("js_callback", pt_callback)
pt_widget.element.html("Page title getter widget")
pt_widget.debugging_display()

In [ ]:
# send the title to the callback
ptelement = pt_widget.element
ptelement.js_callback(ptelement.window.document.title)

In [ ]:
#ptelement.html("Got title: " + repr(pt_callback.called))
#pt_callback.called

In [ ]:
def validate_pt():
    assert type(pt_callback.called) is str 
    ptelement.js_callback(ptelement.window.document.title)
    print("Got title ok.")
    
validators.add_validation(pt_widget, validate_pt)

# Chain of attributes

In [ ]:
ca_widget = jp_proxy_widget.JSProxyWidget()
ca_callback = Callback()
ca_widget.debugging_display()

In [ ]:
ca_widget.js_init("""
    element.html("chain of attributes test widget");
    element.A = {};
    element.A.B = {};
    element.A.B.C = (function() { return callback(); });
    
    // emulate this:
    //element.A.B.C()
""", callback=ca_callback)

In [ ]:
dummy = ca_widget.element.A.B.C()

In [ ]:
def validate_ca():
    assert ca_callback.called is True
    print("Chain of attributes ok.")
    
#validate_ca()
validators.add_validation(ca_widget, validate_ca)

# Chain of functions

In [ ]:
cm_widget = jp_proxy_widget.JSProxyWidget()
cm_callback = Callback()

cm_widget.js_init("""
    element.html("chain of functions test widget");
    element.A = (
        function() {
            return {
                B: (function() { callback() })
            }
        }
    );
    // emulate this:
    //element.A().B()
""", callback=cm_callback)

a = cm_widget.element.A()

ab = cm_widget.element.A().B()

cm_widget.debugging_display()

In [ ]:
def validate_cm():
    assert cm_callback.called is True
    print("Chain of functions ok.")
    
#validate_cm()
validators.add_validation(cm_widget, validate_cm)
a, ab

In [ ]:
"""
try:
    a()
except jp_proxy_widget.StaleFragileJavascriptReference:
    print ("As expected: you can't use a fragile javascript reference outside a chained expression.")
else:
    raise SystemError("Oops: We should have gotten a StaleFragileJavascriptReference raised here.")
    """

# Chained attribute/function

In [ ]:
cam_widget = jp_proxy_widget.JSProxyWidget()
cam_callback = Callback()

cam_widget.js_init("""
    element.html("chain of object with a function attribute");
    element.A = {
        B: function(f) { f("expected value") }
    }
    // emulate this:
    //element.A.B(cam_callback)
""", cam_callback=cam_callback)

#ab = cam_widget.element.A().B()

cam_widget.debugging_display()

In [ ]:
cam_widget.element.A.B(cam_callback)

In [ ]:
def validate_cam():
    assert cam_callback.called == "expected value"
    print("Chain of attribute/function ok.")
    
#validate_cam()
validators.add_validation(cam_widget, validate_cam)

# Chained method of method

In [ ]:
mm_widget = jp_proxy_widget.JSProxyWidget()
mm_callback = Callback()

mm_widget.js_init("""
    element.html("chain method of method");
    class A_class {
        constructor(name) {
            this.name = name;
        };
        get_name() {
            return this.name;
        };
        B(othername) {
            return new B_class(this, othername);
        };
    };
    class B_class {
        constructor(parent, name) {
            this.parent = parent;
            this.name = name;
        };
        send_names(to_callback) {
            return to_callback(this.parent.get_name() + "," + this.name)
        }
    }
    element.A = new A_class("Aname");
    element.callback = mm_callback
    // emulate this:
    //element.A.B("Bname").send_names(element.callback)
""", mm_callback=mm_callback)

mm_widget.debugging_display()

In [ ]:
mm_element = mm_widget.element
mm_element.A.B("Bname").send_names(mm_element.callback)

In [ ]:
def validate_mm():
    assert mm_callback.called == 'Aname,Bname'
    print("Chain method of method ok.")
    
#validate_cam()
validators.add_validation(mm_widget, validate_mm)

#mm_callback.called

In [ ]:
delay_ms = 1000
validators.run_all_in_widget(delay_ms=delay_ms)